In [1]:
%run -i "999-load.py"

/Users/sst/anaconda3/envs/test_xpdstack/lib/python3.8/site-packages/databroker/v1.py:1602: UserWarning: Failed to load config. Falling back to v0.Exception was: Unable to handle metadatastore.module 'databroker.headersource.sqlite'
  warnings.warn(


No config file could be found in the following locations:
/Users/sst/.config/acq
/Users/sst/anaconda3/envs/test_xpdstack/etc/acq
/etc/acq
Loading from packaged simulation configuration
INFO: Initializing the XPD data acquisition environment ...
INFO: area detector has been configured to new acquisition time (time per frame)  = 0.1s
INFO: Reload beamtime objects:

ScanPlans:
0: ct_5
1: ct_0.1
2: ct_1
3: ct_10
4: ct_30
5: ct_60

Samples:
0: poop

{'Verification time': '2021-05-19 15:37:34', 'Verified by': 'st'}



Is this configuration correct? y/n:  y
Please input your initials:  st


INFO: beamtime object has been linked

INFO: Initialized glbl, bt, xrun.
INFO: Publish data to localhost port 5567 with prefix 'raw'.
INFO: Changed home to /Users/sst/acqsim/xpdUser
OK, ready to go.  To continue, follow the steps in the xpdAcqdocumentation at http://xpdacq.github.io/xpdacq


## Configure the frame_acq_time

I suggest using 0.2 s for the frame acquisition time. The default is 0.1 s. The 0.1 s is too short.

In [2]:
glbl["frame_acq_time"] = 0.2

INFO: area detector has been configured to new acquisition time (time per frame)  = 0.2s


## Register the crystream

Usually, it is registered, but there is no harm doing it again. Ask the beamline scientist about the name of the temperature controller.

In [3]:
xpd_configuration['temp_controller'] = cryostream

## Add our samples

In [24]:
bt.list()


ScanPlans:
0: ct_5
1: ct_0.1
2: ct_1
3: ct_10
4: ct_30
5: ct_60

Samples:
0: poop


In [29]:
Sample(bt, {"sample_name": "SrTiO3", "sample_composition": "SrTiO3", "sample_phase": {"Sr": 1, "Ti": 1, "O": 3}})

Sample({'sample_name': 'SrTiO3', 'sample_composition': 'SrTiO3', 'sample_phase': {'Sr': 1, 'Ti': 1, 'O': 3}, 'sa_uid': 'abb61d5a'}, {'bt_piLast': 'Billinge', 'bt_safN': '300000', 'bt_experimenters': [], 'bt_wavelength': 0.1675, 'bt_uid': '8e7631c8'})

In [30]:
bt.list()


ScanPlans:
0: ct_5
1: ct_0.1
2: ct_1
3: ct_10
4: ct_30
5: ct_60

Samples:
0: poop
1: SrTiO3


## Write the plan for one sample as a function

I suggest write one whole plan for one sample as one function so that we can repeatedly use it.

In [33]:
def gen_plan_SrTiO3():
    """Measure one sample."""
    # move to sample position x = 12
    yield from bps.mv(Grid_X, 12.)
    # run the scan, a temperature rampong from 80 to 400 with step 5 and 60s exposure at each point
    # here the pe1c is just a place holder, the detector to use is xpd_configuration['area_det']
    plan = Tramp([pe1c], exposure=60, Tstart=80, Tstop=400, Tstep=5)
    # here we directly inject the sample metadata into the plan
    sample_md = bt.samples.get_md(1)
    plan = bpp.inject_md_wrapper(plan, sample_md)
    print("sample -> {}".format(sample_md.get("sample_name", "")))
    yield from plan
    # move the temperature back to room temperature
    yield from bps.mv(cryostream, 300)

## Preview the plan

Use `summarize_plan` to preview the plan.

In [34]:
from bluesky.simulators import summarize_plan

summarize_plan(gen_plan_SrTiO3())

motor1 -> 12.0
sample -> SrTiO3
acquire_time -> 0.2
images_per_set -> 60.0
INFO: requested exposure time = 60 - > computed exposure time= 60.0
INFO: requested temperature step size = 5 ->computed temperature step size = 5.0
=================================== Open Run ===================================
cs700 -> 80.0
shctl1 -> 60
  Read ['pe1_image', 'cs700']
shctl1 -> 0
cs700 -> 85.0
shctl1 -> 60
  Read ['pe1_image', 'cs700']
shctl1 -> 0
cs700 -> 90.0
shctl1 -> 60
  Read ['pe1_image', 'cs700']
shctl1 -> 0
cs700 -> 95.0
shctl1 -> 60
  Read ['pe1_image', 'cs700']
shctl1 -> 0
cs700 -> 100.0
shctl1 -> 60
  Read ['pe1_image', 'cs700']
shctl1 -> 0
cs700 -> 105.0
shctl1 -> 60
  Read ['pe1_image', 'cs700']
shctl1 -> 0
cs700 -> 110.0
shctl1 -> 60
  Read ['pe1_image', 'cs700']
shctl1 -> 0
cs700 -> 115.0
shctl1 -> 60
  Read ['pe1_image', 'cs700']
shctl1 -> 0
cs700 -> 120.0
shctl1 -> 60
  Read ['pe1_image', 'cs700']
shctl1 -> 0
cs700 -> 125.0
shctl1 -> 60
  Read ['pe1_image', 'cs700']
shctl1 -> 0

## Run the plan

In [12]:
xrun({}, gen_plan_SrTiO3())

INFO: requested exposure time = 60 - > computed exposure time= 60.0
INFO: requested temperature step size = 5 ->computed temperature step size = 5.0
INFO: closing shutter...
INFO: taking dark frame....
INFO: No calibration file found in config_base.
Scan will still keep going on....
INFO: Current filter status
INFO: flt1 : In
INFO: flt2 : Out
INFO: flt3 : Out
INFO: flt4 : Out


+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 15:56:43.8 |
+-----------+------------+
generator count ['86779589'] (scan num: 1)


dark frame complete, update dark dict
opening shutter...
INFO: No calibration file found in config_base.
Scan will still keep going on....
INFO: Current filter status
INFO: flt1 : In
INFO: flt2 : Out
INFO: flt3 : Out
INFO: flt4 : Out


+-----------+------------+----------------+-------------+
|   seq_num |       time | cs700_setpoint | temperature |
+-----------+------------+----------------+-------------+
|         1 | 15:56:45.0 |   

('86779589-15b0-49bc-9ef6-fa7a141ea048',
 '5076f031-7d6c-42fd-a5b0-5c0492c9305e')

## Chain the plan together

We have another sample and we would like to first measure sample SrTiO3 and then measure BaTiO3. We use `pchain` to chain the two plans together. Note that we use `gen_plan_SrTiO3()` here to create the generator and chain the generators together instead of chaining functions together.

In [37]:
def gen_plan_BaTiO3():
    """Measure another sample."""
    # move to sample position x = 12
    yield from bps.mv(Grid_X, 15.)
    # run the scan, a temperature rampong from 80 to 400 with step 5 and 60s exposure at each point
    # here the pe1c is just a place holder, the detector to use is xpd_configuration['area_det']
    plan = Tramp([pe1c], exposure=60, Tstart=80, Tstop=400, Tstep=5)
    # here we directly inject the sample metadata into the plan
    sample_md = bt.samples.get_md(1)
    plan = bpp.inject_md_wrapper(plan, sample_md)
    print("sample -> {}".format(sample_md.get("sample_name", "")))
    yield from plan
    # move the temperature back to room temperature
    yield from bps.mv(cryostream, 300)

In [27]:
import bluesky.preprocessors as bpp


chained_plan1 = bpp.pchain(gen_plan_SrTiO3(), gen_plan_BaTiO3())

Or we can also define another function like this.

In [19]:
def gen_two_plans():
    yield from gen_plan_SrTiO3()
    yield from gen_plan_BaTiO3()

In [20]:
chained_plan2 = gen_two_plans()

## Check the run

In [21]:
latest_run = db[-1]

In [22]:
latest_run.start

{'uid': '5076f031-7d6c-42fd-a5b0-5c0492c9305e',
 'time': 1621454204.066535,
 'versions': {'ophyd': '1.6.1', 'bluesky': '1.6.7'},
 'beamline_id': '28-id-2',
 'group': 'sim',
 'facility': 'sim',
 'beamline_config': {'Verification time': '2021-05-19 15:43:39',
  'Verified by': 'st'},
 'bt_piLast': 'Billinge',
 'bt_safN': '300000',
 'bt_experimenters': [],
 'bt_wavelength': 0.1675,
 'bt_uid': '8e7631c8',
 'scan_id': 2,
 'plan_type': 'generator',
 'plan_name': 'scan',
 'detectors': ['pe1_image'],
 'motors': ['cs700'],
 'num_points': 65,
 'num_intervals': 64,
 'plan_args': {'detectors': ["SynSignalWithRegistry(name='pe1_image', value=array([[0., 0., 0., ..., 0., 0., 0.],\n       [0., 0., 0., ..., 0., 0., 0.],\n       [0., 0., 0., ..., 0., 0., 0.],\n       ...,\n       [0., 0., 0., ..., 0., 0., 0.],\n       [0., 0., 0., ..., 0., 0., 0.],\n       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), timestamp=1621453414.273301)"],
  'num': 65,
  'args': ["SynAxis(prefix='', name='cs700', read_attrs=['readback', 'setpoint'], configuration_attrs=['velocity', 'acceleration'])",
   80,
   400],
  'per_step': '<function shutter_step at 0x7fa1a5ae3670>'},
 'hints': {'dimensions': [[['temperature'], 'primary']]},
 'plan_pattern': 'inner_product',
 'plan_pattern_module': 'bluesky.plan_patterns',
 'plan_pattern_args': {'num': 65,
  'args': ["SynAxis(prefix='', name='cs700', read_attrs=['readback', 'setpoint'], configuration_attrs=['velocity', 'acceleration'])",
   80,
   400]},
 'sp_time_per_frame': 0.2,
 'sp_num_frames': 300.0,
 'sp_requested_exposure': 60,
 'sp_computed_exposure': 60.0,
 'sp_type': 'Tramp',
 'sp_startingT': 80,
 'sp_endingT': 400,
 'sp_requested_Tstep': 5,
 'sp_computed_Tstep': 5.0,
 'sp_Nsteps': 65,
 'sp_uid': '98fb4063-9a30-4ab7-b723-4ad30ad3493a',
 'sp_plan_name': 'Tramp',
 'sc_dk_field_uid': '86779589-15b0-49bc-9ef6-fa7a141ea048',
 'detector_calibration_client_uid': '4ed14bab-d23b-467c-9f32-20bc09538598',
 'xpdacq_md_version': 0.1,
 'analysis_stage': 'raw',
 'filter_positions': {'flt1': 'In',
  'flt2': 'Out',
  'flt3': 'Out',
  'flt4': 'Out'}}

In [23]:
latest_run.stop

{'run_start': '5076f031-7d6c-42fd-a5b0-5c0492c9305e',
 'time': 1621454257.0665238,
 'uid': '509988ff-db9d-4a7a-89b9-2ea35db1ba88',
 'exit_status': 'success',
 'reason': '',
 'num_events': {'primary': 65}}